In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.image import imread
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import utils
from tensorflow.keras import models
from sklearn.metrics import classification_report,confusion_matrix
import tensorflow as tf
# to share the GPU resources for multiple sessions
from tensorflow.compat.v1.keras.backend import set_session
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True # dynamically grow the memory used on the GPU
config.log_device_placement = True # to log device placement (on which device the operation ran)
sess = tf.compat.v1.Session(config=config)
set_session(sess)

%matplotlib inline

**Dataset Download Link**
[https://drive.google.com/file/d/1ULKLK_R0qsLho6PzfUIOwTXOStk3w5jd/view?usp=sharing](https://drive.google.com/file/d/1ULKLK_R0qsLho6PzfUIOwTXOStk3w5jd/view?usp=sharing)

In [ ]:
# Upload this file to your google drive

In [ ]:
# for Google Colab
drive.mount('/content/drive')

In [ ]:
# for Google Colab
!tar --skip-old-files -xvf '/content/drive/MyDrive/Dataset/cell_images.tar.xz' -C '/content/drive/MyDrive/Dataset/'

In [ ]:
# for Google Colab
my_data_dir = '/content/drive/MyDrive/Dataset/cell_images'

In [ ]:
# for college server
my_data_dir = '/home/ailab/hdd/dataset/cell_images'

In [ ]:
os.listdir(my_data_dir)

In [ ]:
test_path = my_data_dir+'/test/'
train_path = my_data_dir+'/train/'

In [ ]:
os.listdir(train_path)

In [ ]:
len(os.listdir(train_path+'/uninfected/'))

In [ ]:
len(os.listdir(train_path+'/parasitized/'))

In [ ]:
os.listdir(train_path+'/parasitized')[0]

In [ ]:
para_img= imread(train_path+
                 '/parasitized/'+
                 os.listdir(train_path+'/parasitized')[0])

In [ ]:
plt.imshow(para_img)

In [ ]:
# Checking the image dimensions
dim1 = []
dim2 = []
for image_filename in os.listdir(test_path+'/uninfected'):
    img = imread(test_path+'/uninfected'+'/'+image_filename)
    d1,d2,colors = img.shape
    dim1.append(d1)
    dim2.append(d2)

In [ ]:
sns.jointplot(x=dim1,y=dim2)

In [ ]:
image_shape = (130,130,3)

In [ ]:
help(ImageDataGenerator)

In [ ]:
image_gen = ImageDataGenerator(rotation_range=20, # rotate the image 20 degrees
                               width_shift_range=0.10, # Shift the pic width by a max of 5%
                               height_shift_range=0.10, # Shift the pic height by a max of 5%
                               rescale=1/255, # Rescale the image by normalzing it.
                               shear_range=0.1, # Shear means cutting away part of the image (max 10%)
                               zoom_range=0.1, # Zoom in by 10% max
                               horizontal_flip=True, # Allo horizontal flipping
                               fill_mode='nearest' # Fill in missing pixels with the nearest filled value
                              )

In [ ]:
image_gen.flow_from_directory(train_path)

In [ ]:
image_gen.flow_from_directory(test_path)

In [ ]:
model = models.Sequential()
# Write your code here

model.compile(loss='',
              optimizer='',
              metrics=[''])

In [ ]:
model.summary()

In [ ]:
batch_size = 16

In [ ]:
help(image_gen.flow_from_directory)

In [ ]:
train_image_gen = image_gen.flow_from_directory(train_path,
                                               target_size=image_shape[:2],
                                                color_mode='rgb',
                                               batch_size=batch_size,
                                               class_mode='binary')

In [ ]:
train_image_gen.batch_size

In [ ]:
len(train_image_gen.classes)

In [ ]:
train_image_gen.total_batches_seen

In [ ]:
test_image_gen = image_gen.flow_from_directory(test_path,
                                               target_size=image_shape[:2],
                                               color_mode='rgb',
                                               batch_size=batch_size,
                                               class_mode='binary',shuffle=False)

In [ ]:
train_image_gen.class_indices

In [ ]:
results = model.fit(train_image_gen,epochs=20,
                              validation_data=test_image_gen
                             )

In [ ]:
model.save('cell_model.h5')

In [ ]:
losses = pd.DataFrame(model.history.history)

In [ ]:
losses[['loss','val_loss']].plot()

In [ ]:
model.metrics_names

In [ ]:
model.evaluate(test_image_gen)

In [ ]:
pred_probabilities = model.predict(test_image_gen)

In [ ]:
test_image_gen.classes

In [ ]:
predictions = pred_probabilities > 0.5

In [ ]:
print(classification_report(test_image_gen.classes,predictions))

In [ ]:
confusion_matrix(test_image_gen.classes,predictions)